# Vector Matching Test Bench 

Se crean archivos con números en Fixed Point aleatorios para y se opera con ellos, para obtener patrones de entradas aleatorias para el test bench de los módulos y patrones conocidos de salidas para realizar un vector matching 

In [3]:
from tool._fixedInt import *    # Modulo de punto fijo de Fulgor 
from tool.DSPtools import * 
import numpy as np

## Vectores patrones
* Definimos una función que se encargue de crear las entradas aleatorias de un formato determinado 

In [6]:
def randFixedPointArray(intWidth=4,fractWidth=2,
                        signedMode = 'S',roundMode='trunc',
                        saturateMode='saturate',N=1000):
    
    Fp = DeFixedInt(intWidth,fractWidth,signedMode,roundMode,saturateMode)

    Rv = Fp.Range()
    values = np.random.uniform(Rv[0],Rv[1],N) #Dentro del rango como para no sature y sea 
                                                #Varios numeros iguales (maximo o minimo), 
                          

    return arrayFixedInt(intWidth, fractWidth, values, signedMode, 
                         roundMode, saturateMode)

* Definimos los N valoers de entradas: 
     * A : S(16,14) 
     * B : S(12.11) 


In [7]:
A = randFixedPointArray(16,14,'S')
B = randFixedPointArray(12,11,'S')

 ### Suma
 * Realizamos la sumas  **Full Resolution**

In [8]:
SumaFullRes = A + B 

 * Ahora operamos con este valor para obtener los diferentes formatos
  
     * S(11.10) con overflow y truncado
     * S(11.10) con saturacion y truncado
     * S(9.8) con saturación y redondeo
     
 <span style="color:red">**Nota**</span> : Se modificó la función *arrayFixedInt()* para que pueda tomar un valor de tipo *DeFixedInt* y reconvertirlo al formato especificado. 
 
 De esta forma se puede **reconvertir arreglos** directamente.  
 Funciona para el caso de que se aplique truncado, o cualquier tipo de recorte. No se probó si se debe agregar bits, pero eso depende de la clase misma ya implementada.      

In [10]:
SumOvTrunc = arrayFixedInt(11,10,SumaFullRes,'S','trunc','wrap')
SumSatTrunc = arrayFixedInt(11,10,SumaFullRes,'S','trunc','saturate')
SumSatRound = arrayFixedInt(9,8,SumaFullRes,'S','round','saturate')

*  <span style="color:red">**Comprabamos a grandes rasgos** </span> 

In [11]:
V1  = SumOvTrunc[4]  
V2=DeFixedInt(11,10,'S','trunc','wrap')
V2.assign(A[4] + B[4])

print(" Version 1 = ", V1,
      "\n Version 2 = ", V2 )

V1  = SumSatTrunc[4]  
V2=DeFixedInt(11,10,'S','trunc','saturate')
V2.assign(A[4] + B[4])

print("\n Version 1 = ", V1,
      "\n Version 2 = ", V2 )


V1  = SumSatRound[4]  
V2=DeFixedInt(9,8,'S','round','saturate')
V2.assign(A[4] + B[4])

print("\n Version 1 = ", V1,
      "\n Version 2 = ", V2 )



 Version 1 =  <669 (0.653) S(11,10)> 
 Version 2 =  <669 (0.653) S(11,10)>

 Version 1 =  <-1024 (-1.000) S(11,10)> 
 Version 2 =  <-1024 (-1.000) S(11,10)>

 Version 1 =  <-256 (-1.000) S(9,8)> 
 Version 2 =  <-256 (-1.000) S(9,8)>


* Ahora, guardamos estos vectores de entrada A y B, y de salida Sum*Format* en archivos separados para poder trabajarlos en el testbench. 



In [12]:
def SaveBytesTbFile(FpValues,FileName):
    TbFile  = open(FileName,'w')
    for FpNum in FpValues:
    
        TbBin =bin(FpNum.intvalue)[2:]      #"0b" no me interesa 
        TbBin = TbBin.zfill(FpNum.width)    #Aveces bin() devuelve menos bits
        TbBin = str(TbBin)
        TbFile.write(TbBin)
        TbFile.write("\n")
    TbFile.close()

In [13]:
SaveBytesTbFile(A,"./A.in")
SaveBytesTbFile(B,"./B.in")
SaveBytesTbFile(SumaFullRes,"./PatternVec/SumFullRes.o")
SaveBytesTbFile(SumOvTrunc,"./PatternVec/SumOvTrunc.o")
SaveBytesTbFile(SumSatTrunc,"./PatternVec/SumSatTrunc.o")
SaveBytesTbFile(SumSatRound ,"./PatternVec/SumSatRound.o")

Si se quiere seguir el  [ejemplo](https://github.com/matias-dogliani/diseno_digital/blob/master/Tp3/rlt_ejemplo/vectors/switch.out) del formato de testbench de trabajo en anteriores trabajos prácicos se puede insertar el separador de tabulador entre cada bit. Aunque en este caso no se usa, se puede utilizar el mismo print() para hacerlo facilmente

### Multiplicación


## Comparación 

* Comparamos, ahora, los archivos generados en python, con los registros generados en verilog. Podemos hacerlo simplemente comparando las lineas de ambos archivos, o generado un archivo de registro en caso de que exista un error. 

```
TbBin = open("./TbVec/SumFullResV.out",'r').readlines()
PyBin = open("./PatternVec/SumFullRes.o",'r').readlines()
for i,resultado in enumerate(PyBin): 

    if(resultado != TbBin[i]):
        print("Diferentes : Py =",resultado,
             "Verilog =  ",TbBin[i])
```

* Se crean funciones de comparación y de creación del archivo log 

In [2]:
def CompareTbBin(BinPyFileName, TbBinFileName): 
   
    error = 0
    PatternValue = []
    CorruptedValue = [] 
    
    TbBin = open(TbBinFileName,'r').readlines()
    PyBin = open(BinPyFileName,'r').readlines()
    
    for i,res in enumerate(PyBin): 
        if(res != TbBin[i]):
            error +=1 
            PatternValue.append(res[:-1]) #Saco el \n
            CorruptedValue.append(TbBin[i])

    errorRate = error/i if error else 0
    
    return errorRate,error,PatternValue,CorruptedValue 


In [1]:
def LogFileGen (error,errorRate,PatternValue,CorruptedValue,FileName): 
    if (er): 
        logFile= open(FileName,'w')
        print(error,errorRate,sep=" ",file=logFile)
        print("Python", "TestBench",sep = ",",file=logFile)
        for i,Value in enumerate(PatternValue):
            print(i,Value, CorruptedValue[i], sep=",",file=logFile)  
        logFile.close()

### Suma
* Comparamos los resultados de suma **Full Resolution**

In [14]:
[er,erR,PValue,CValue] = CompareTbBin("./TbVec/SumFullResV.out",
                                      "./PatternVec/SumFullRes.o") 
print("Error en Full Resolution: ", er)
if (er): 
    LogFileGen(er,erR,PValue,CValue,"./logs/FullRes.log")

Error en Full Resolution:  0


* Comparamos los resultados de suma **S(11,10) truncado y overflow**

In [15]:
[er,erR,PValue,CValue] = CompareTbBin("./TbVec/SumOvTruncV.out",
                                      "./PatternVec/SumOvTrunc.o")
print("Error en Overflow y Truncado: ", er)
if (er): 
    LogFileGen(er,erR,PValue,CValue,"./logs/SumOvTrunc.log")

Error en Overflow y Truncado:  0


* Comparamos los resultados de suma **S(11,10) truncado y saturación**

In [16]:
[er,erR,PValue,CValue] = CompareTbBin("./TbVec/SumSatTruncV.out",
                                      "./PatternVec/SumSatTrunc.o") 
print("Error en Saturación y Truncado: ", er)
if (er): 
    LogFileGen(er,erR,PValue,CValue,"./logs/SumSatTrunc.log")

Error en Saturación y Truncado:  0


* Comparamos los resultados de suma **S(11,10) redondeo y saturación**

In [17]:
[er,erR,PValue,CValue] = CompareTbBin("./TbVec/SumSatRoundV.out",
                                      "./PatternVec/SumSatRound.o")
print("Error en Saturación y Redondeo: ", er)
if (er): 
    LogFileGen(er,erR,PValue,CValue,"./logs/SumSatRound.log")

Error en Saturación y Redondeo:  0.19119119119119118


### Multiplicación

* Creamos los arreglos con los vectores patrones

In [97]:
A = randFixedPointArray(8,6,'S')
B = randFixedPointArray(12,11,'S')
SaveBytesTbFile(A,"./A.in")
SaveBytesTbFile(B,"./B.in")

In [98]:
MulFullRes = A*B 
MulOvTrunc = arrayFixedInt(12,11,MulFullRes,'S','trunc','wrap')
MulSatTrunc = arrayFixedInt(12,11,MulFullRes,'S','trunc','saturate')
MulSatRound = arrayFixedInt(10,9,MulFullRes,'S','round','saturate')

 * Guardamos en un archivo estos vectores calculados

In [99]:
SaveBytesTbFile(MulFullRes,"./PatternVec/MulFullRes.o")
SaveBytesTbFile(MulOvTrunc,"./PatternVec/MulOvTrunc.o")
SaveBytesTbFile(MulSatTrunc,"./PatternVec/MulSatTrunc.o")
SaveBytesTbFile(MulSatRound ,"./PatternVec/MulSatRound.o")

In [105]:
print(bin(A[0].intvalue), bin(B[0].intvalue))
print(MulFullRes[0])
print(bin(MulFullRes[0].intvalue))
print(MulOvTrunc[0])
print(bin(MulOvTrunc[0].intvalue))
print(MulSatTrunc[0])
print(bin(MulSatTrunc[0].intvalue))
print(MulSatRound[0])
print(bin(MulSatRound[0].intvalue))

0b10001100 0b1011011001
<-84564 (-0.645) S(20,17)>
0b11101011010110101100
<-1322 (-0.646) S(12,11)>
0b101011010110
<-1322 (-0.646) S(12,11)>
0b101011010110
<-330 (-0.645) S(10,9)>
0b1010110110


* Comparamos vectores 

In [5]:
[er,erR,PValue,CValue] = CompareTbBin("./TbVec/MulFullResV.out",
                                      "./PatternVec/MulFullRes.o") 
print("Error en Full Resolution: ", er)
if (er): 
    LogFileGen(er,erR,PValue,CValue,"./logs/FullRes.log")
  
    
[er,erR,PValue,CValue] = CompareTbBin("./TbVec/MulOvTruncV.out",
                                      "./PatternVec/MulOvTrunc.o") 
print("Error en Overflow y Truncado: ", er)
if (er): 
    LogFileGen(er,erR,PValue,CValue,"./logs/MulOvTrunc.log")
    

[er,erR,PValue,CValue] = CompareTbBin("./TbVec/MulSatTruncV.out",
                                      "./PatternVec/MulSatTrunc.o")
print("Error en Saturación y Truncado: ", er)
if (er): 
    LogFileGen(er,erR,PValue,CValue,"./logs/MulSatTrunc.log")


[er,erR,PValue,CValue] = CompareTbBin("./TbVec/MulSatRoundV.out",
                                      "./PatternVec/MulSatRound.o")
print("Error en Saturación y Redondeo: ", er)
if (er): 
    LogFileGen(er,erR,PValue,CValue,"./logs/MulSatRound.log")

Error en Full Resolution:  0
Error en Overflow y Truncado:  0
Error en Saturación y Truncado:  0
Error en Saturación y Redondeo:  0.3233233233233233
